In [18]:
import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install scikit-learn
import numpy as np, keras, librosa, tensorflow, scipy

In [19]:
import os

In [20]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    if (max_pad_len>mfcc.shape[1]):
        pad_width = max_pad_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)),mode='constant')
    else:
        mfcc = mfcc[:,:max_pad_len]
    return mfcc

In [21]:
import numpy as np
DATA_PATH = "/home/aaa/aatresh/Audio Processing/data/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, keras.utils.to_categorical(label_indices)

In [22]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels,_,_= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        
        wavfiles = [path+label+'/'+wavfile for wavfile in os.listdir(path+'/'+label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
            
        np.save(label+'.npy',mfcc_vectors)

In [23]:
from sklearn.model_selection import train_test_split
def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices,_ = get_labels(DATA_PATH)
    
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0] == len(y)
    
    return train_test_split(X, y, test_size=(1-split_ratio),random_state=random_state, shuffle=True)

In [24]:
save_data_to_array()

In [25]:
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = keras.utils.to_categorical(y_train)
y_test_hot = keras.utils.to_categorical(y_test)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu',input_shape=(20,11,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(X_train, y_train_hot,batch_size=100, epochs=300, verbose=1, validation_data=(X_test, y_test_hot))


Train on 5746 samples, validate on 3831 samples
Epoch 1/300
5746/5746 [==============================] - 1s 204us/step - loss: 10.6827 - acc: 0.2973 - val_loss: 8.6960 - val_acc: 0.4176
Epoch 2/300
5746/5746 [==============================] - 1s 159us/step - loss: 8.0706 - acc: 0.4206 - val_loss: 3.5665 - val_acc: 0.6045
Epoch 3/300
5746/5746 [==============================] - 1s 179us/step - loss: 2.1060 - acc: 0.4843 - val_loss: 1.0771 - val_acc: 0.5516
Epoch 4/300
5746/5746 [==============================] - 1s 140us/step - loss: 1.1596 - acc: 0.5211 - val_loss: 0.9412 - val_acc: 0.5946
Epoch 5/300
5746/5746 [==============================] - 1s 166us/step - loss: 1.0654 - acc: 0.5670 - val_loss: 0.8421 - val_acc: 0.6742
Epoch 6/300
5746/5746 [==============================] - 1s 162us/step - loss: 0.9392 - acc: 0.6077 - val_loss: 0.7469 - val_acc: 0.7204
Epoch 7/300
5746/5746 [==============================] - 1s 169us/step - loss: 0.8748 - acc: 0.6493 - val_loss: 0.7086 - val_acc:

5746/5746 [==============================] - 1s 150us/step - loss: 0.2124 - acc: 0.9163 - val_loss: 0.2950 - val_acc: 0.8938
Epoch 61/300
5746/5746 [==============================] - 1s 151us/step - loss: 0.2331 - acc: 0.9121 - val_loss: 0.2884 - val_acc: 0.8945
Epoch 62/300
5746/5746 [==============================] - 1s 147us/step - loss: 0.2272 - acc: 0.9139 - val_loss: 0.2785 - val_acc: 0.8943
Epoch 63/300
5746/5746 [==============================] - 1s 151us/step - loss: 0.2174 - acc: 0.9156 - val_loss: 0.2999 - val_acc: 0.8862
Epoch 64/300
5746/5746 [==============================] - 1s 149us/step - loss: 0.2204 - acc: 0.9168 - val_loss: 0.2792 - val_acc: 0.8966
Epoch 65/300
5746/5746 [==============================] - 1s 131us/step - loss: 0.2158 - acc: 0.9194 - val_loss: 0.2915 - val_acc: 0.8951
Epoch 66/300
5746/5746 [==============================] - 1s 125us/step - loss: 0.2050 - acc: 0.9238 - val_loss: 0.3074 - val_acc: 0.8901
Epoch 67/300
5746/5746 [=======================

5746/5746 [==============================] - 1s 151us/step - loss: 0.1242 - acc: 0.9534 - val_loss: 0.3208 - val_acc: 0.9024
Epoch 120/300
5746/5746 [==============================] - 1s 147us/step - loss: 0.1230 - acc: 0.9539 - val_loss: 0.3000 - val_acc: 0.9045
Epoch 121/300
5746/5746 [==============================] - 1s 154us/step - loss: 0.1237 - acc: 0.9565 - val_loss: 0.3227 - val_acc: 0.9019
Epoch 122/300
5746/5746 [==============================] - 1s 167us/step - loss: 0.1093 - acc: 0.9617 - val_loss: 0.3375 - val_acc: 0.8945
Epoch 123/300
5746/5746 [==============================] - 1s 160us/step - loss: 0.1258 - acc: 0.9541 - val_loss: 0.3103 - val_acc: 0.9005
Epoch 124/300
5746/5746 [==============================] - 1s 154us/step - loss: 0.1091 - acc: 0.9607 - val_loss: 0.3391 - val_acc: 0.8974
Epoch 125/300
5746/5746 [==============================] - 1s 149us/step - loss: 0.1280 - acc: 0.9537 - val_loss: 0.3271 - val_acc: 0.8964
Epoch 126/300
5746/5746 [================

5746/5746 [==============================] - 1s 151us/step - loss: 0.0908 - acc: 0.9666 - val_loss: 0.3276 - val_acc: 0.9089
Epoch 179/300
5746/5746 [==============================] - 1s 152us/step - loss: 0.1091 - acc: 0.9628 - val_loss: 0.3311 - val_acc: 0.9063
Epoch 180/300
5746/5746 [==============================] - 1s 151us/step - loss: 0.0915 - acc: 0.9676 - val_loss: 0.3458 - val_acc: 0.9032
Epoch 181/300
5746/5746 [==============================] - 1s 146us/step - loss: 0.1031 - acc: 0.9621 - val_loss: 0.3637 - val_acc: 0.9032
Epoch 182/300
5746/5746 [==============================] - 1s 151us/step - loss: 0.1001 - acc: 0.9640 - val_loss: 0.3352 - val_acc: 0.9076
Epoch 183/300
5746/5746 [==============================] - 1s 149us/step - loss: 0.0889 - acc: 0.9683 - val_loss: 0.3288 - val_acc: 0.9013
Epoch 184/300
5746/5746 [==============================] - 1s 151us/step - loss: 0.0898 - acc: 0.9690 - val_loss: 0.3385 - val_acc: 0.9026
Epoch 185/300
5746/5746 [================

5746/5746 [==============================] - 1s 150us/step - loss: 0.0880 - acc: 0.9699 - val_loss: 0.3366 - val_acc: 0.9094
Epoch 238/300
5746/5746 [==============================] - 1s 150us/step - loss: 0.0803 - acc: 0.9708 - val_loss: 0.3255 - val_acc: 0.9086
Epoch 239/300
5746/5746 [==============================] - 1s 150us/step - loss: 0.0806 - acc: 0.9704 - val_loss: 0.3639 - val_acc: 0.9066
Epoch 240/300
5746/5746 [==============================] - 1s 152us/step - loss: 0.0774 - acc: 0.9741 - val_loss: 0.3583 - val_acc: 0.9081
Epoch 241/300
5746/5746 [==============================] - 1s 154us/step - loss: 0.0839 - acc: 0.9720 - val_loss: 0.3570 - val_acc: 0.9032
Epoch 242/300
5746/5746 [==============================] - 1s 148us/step - loss: 0.0859 - acc: 0.9715 - val_loss: 0.3483 - val_acc: 0.9094
Epoch 243/300
5746/5746 [==============================] - 1s 151us/step - loss: 0.0817 - acc: 0.9722 - val_loss: 0.3295 - val_acc: 0.9066
Epoch 244/300
5746/5746 [================

5746/5746 [==============================] - 1s 152us/step - loss: 0.0685 - acc: 0.9774 - val_loss: 0.3670 - val_acc: 0.9066
Epoch 297/300
5746/5746 [==============================] - 1s 157us/step - loss: 0.0710 - acc: 0.9763 - val_loss: 0.3816 - val_acc: 0.9115
Epoch 298/300
5746/5746 [==============================] - 1s 152us/step - loss: 0.0681 - acc: 0.9755 - val_loss: 0.3500 - val_acc: 0.9105
Epoch 299/300
5746/5746 [==============================] - 1s 149us/step - loss: 0.0703 - acc: 0.9769 - val_loss: 0.3847 - val_acc: 0.9066
Epoch 300/300
5746/5746 [==============================] - 1s 152us/step - loss: 0.0746 - acc: 0.9741 - val_loss: 0.3796 - val_acc: 0.9052


In [51]:
import socket
import numpy as np
from pygame import mixer
import time


try:
        mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print ("Socket created")

except socket.error:
        print ("Socket connection failed")

port = 8888
try:
        mySock.connect(("192.168.43.195", port))
        print ("Connected to RPi!")
except socket.error:
        print ("Not able to connect")



"""
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "out.wav"
while True:
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        print("* recording")

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        print("* done recording")

        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        recording = AudioSegment.from_wav("./out.wav")
        recording+=20
        recording.export("./out.wav", format="wav")

        print("Before the sleep statement")
        mixer.init(frequency=44100)
        mixer.music.load("./out.wav")
        mixer.music.play()
        time.sleep(5)
        print("After the sleep statement")
        
"""

filenames=os.listdir("/home/aaa/aatresh/Audio Processing/New folder/")

while(1):
 
    for f in filenames:
        sample = wav2mfcc("/home/aaa/aatresh/Audio Processing/New folder/"+f)
        sample_reshaped = sample.reshape(1,20,11,1)
        message = get_labels("/home/aaa/aatresh/Audio Processing/data/")[0][np.argmax(model.predict(sample_reshaped))]   

        mixer.init(frequency=44100)
        mixer.music.load("/home/aaa/aatresh/Audio Processing/New folder/"+f)
        mixer.music.play()
        
        time.sleep(2)
        
        print(message)
        mySock.send(message.encode())
        data = mySock.recv(1024).decode()
        print ("Received from server: " + data)
        
        

mySock.close()


KeyboardInterrupt: 